In [9]:
import math
import numpy as np
import csv
from sklearn.preprocessing import StandardScaler

class MySoftmaxRegression:
    def __init__(self, learning_rate=0.05, epochs=10000):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.w = None  
        self.free = None 

    # practic probabiltae sa fie una dintre cele 3 cat la suta sa fie unu 2 sau 0  
    @staticmethod
    def softmax(z):
        result = []
        for r in z:
            max_val = max(r)
            exp_row = [math.exp(val - max_val) for val in r]
            sum_exp = sum(exp_row)
            result.append([val / sum_exp for val in exp_row])
        return result

    @staticmethod
    def transpose(matrix):
        return [[matrix[i][j] for i in range(len(matrix))] for j in range(len(matrix[0]))]

    @staticmethod
    def mat_vec_dot(matrix, vector):
        result = []
        for r in matrix:
            s = 0
            for i in range(len(vector)):
                s += r[i] * vector[i]
            result.append(s)
        return result

    @staticmethod
    def mul_matrix(a, b):
        result = []
        for i in range(len(a)):
            r = []
            for j in range(len(b[0])):
                val = sum(a[i][k] * b[k][j] for k in range(len(b)))
                r.append(val)
            result.append(r)
        return result

    @staticmethod
    def subtract_matrix(a, b):
        return [[a[i][j] - b[i][j] for j in range(len(a[0]))] for i in range(len(a))]

    @staticmethod
    def scalar_divide_matrix(matrix, s):
        return [[val / s for val in r] for r in matrix]

    @staticmethod
    def scalar_subtract_matrix(a, b, lr):
        return [[a[i][j] - lr * b[i][j] for j in range(len(a[0]))] for i in range(len(a))]

    @staticmethod
    def add_free(matrix, free):
        return [[matrix[i][j] + free[0][j] for j in range(len(free[0]))] for i in range(len(matrix))]

    @staticmethod
    def to_bin(y, num_classes):
        return [[1 if i == label else 0 for i in range(num_classes)] for label in y]

    def fit(self, x, y):
        n_samples = len(x)
        n_features = len(x[0])
        values = max(y) + 1

        self.w = [[0.0 for _ in range(values)] for _ in range(n_features)]
        self.free = [[0.0 for _ in range(values)]]
        y_one_hot = self.to_bin(y, values)

        for _ in range(self.epochs):
            logits = self.add_free(self.mul_matrix(x, self.w), self.free)
            probs = self.softmax(logits)
            error = self.subtract_matrix(probs, y_one_hot)

            x_t = self.transpose(x)
            dw = self.mul_matrix(x_t, error)
            dw = self.scalar_divide_matrix(dw, n_samples)

            db = [[sum(error[i][j] for i in range(n_samples)) / n_samples for j in range(values)]]

            self.w = self.scalar_subtract_matrix(self.w, dw, self.learning_rate)
            self.free = self.scalar_subtract_matrix(self.free, db, self.learning_rate)
            
        return self.free, self.w

    def predict(self, x):
        logits = self.add_free(self.mul_matrix(x, self.w), self.free)
        probs = self.softmax(logits)
        predictions = []
        for r in probs:
            max_index = 0
            max_val = r[0]
            for i in range(1, len(r)):
                if r[i] > max_val:
                    max_val = r[i]
                    max_index = i
            predictions.append(max_index)
        return predictions


def load_data(file_name):
    data = []
    with open(file_name) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for r in csv_reader:
            sepal_length = float(r[0])
            sepal_width = float(r[1])
            petal_length = float(r[2])
            petal_width = float(r[3]) 
            if r[4] == 'Iris-setosa':
                label = 0
            elif r[4] == 'Iris-versicolor':
                label = 1
            else:  
                label = 2
            features = [sepal_length, sepal_width, petal_length, petal_width]
            data.append([features, label])
            
    x = [d[0] for d in data]
    y = [d[1] for d in data]
    return x, y


file = "../data/iris.data"
inputs, outputs = load_data(file)

feature1 = [ex[0] for ex in inputs]
feature2 = [ex[1] for ex in inputs]
feature3 = [ex[2] for ex in inputs]
feature4 = [ex[3] for ex in inputs]

np.random.seed(5)
indexes = [i for i in range(len(inputs))]
trainSample = np.random.choice(indexes, int(0.8 * len(inputs)), replace=False)
testSample = [i for i in indexes if not i in trainSample]

trainInputs = [inputs[i] for i in trainSample]
trainOutputs = [outputs[i] for i in trainSample]
testInputs = [inputs[i] for i in testSample]
testOutputs = [outputs[i] for i in testSample]

scaler = StandardScaler()

trainInputs = scaler.fit_transform(trainInputs)
testInputs = scaler.transform(testInputs)

feature1train = [ex[0] for ex in trainInputs]
feature2train = [ex[1] for ex in trainInputs]
feature3train = [ex[2] for ex in trainInputs]
feature4train = [ex[3] for ex in trainInputs]
feature1test = [ex[0] for ex in testInputs]
feature2test = [ex[1] for ex in testInputs]
feature3test = [ex[2] for ex in testInputs]
feature4test = [ex[3] for ex in testInputs]

regressor = MySoftmaxRegression()
w0, w = regressor.fit(trainInputs, trainOutputs)
print(f'the learnt model: f(x) = {w0} + {w[0]} * x1 + {w[1]} * x2+ {w[2]} * x3 + {w[3]} * x4')
print("Biasuri (free):", regressor.free)
print("Greutăți (w):")
for i, row in enumerate(regressor.w):
    print(f"  x{i}: {row}")
computedTestOutputs = regressor.predict(testInputs)
new_datail = scaler.transform([[5.35,3.85,1.25,0.4]])
pred = regressor.predict(new_datail)
print("Probabilități pentru fiecare clasă:", pred[0])

print("Noua floare este din specia:", "Iris-setosa" if pred == 0 else 
      "Iris-versicolor" if pred == 1 else "Iris-virginica")


the learnt model: f(x) = [[-0.33263330217448506, 3.8841747643681894, -3.5515414621937054]] + [np.float64(-1.8108150348896292), np.float64(1.1851075198031353), np.float64(0.6257075150865025)] * x1 + [np.float64(2.276341190800492), np.float64(-0.6133260171729873), np.float64(-1.6630151736275203)] * x2+ [np.float64(-3.2293291476294304), np.float64(-0.9417360026766695), np.float64(4.171065150306081)] * x3 + [np.float64(-3.1381978453947235), np.float64(-1.3545164103929324), np.float64(4.492714255787706)] * x4
Biasuri (free): [[-0.33263330217448506, 3.8841747643681894, -3.5515414621937054]]
Greutăți (w):
  x0: [np.float64(-1.8108150348896292), np.float64(1.1851075198031353), np.float64(0.6257075150865025)]
  x1: [np.float64(2.276341190800492), np.float64(-0.6133260171729873), np.float64(-1.6630151736275203)]
  x2: [np.float64(-3.2293291476294304), np.float64(-0.9417360026766695), np.float64(4.171065150306081)]
  x3: [np.float64(-3.1381978453947235), np.float64(-1.3545164103929324), np.float6